In [ ]:
import pandas as pd

link_dist_vs_lanes = pd.read_parquet('/content/la_link_dist_vs_lanes_estimation_dataset_v1.parquet')
link_dist_vs_lanes.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/la_link_dist_vs_lanes_estimation_dataset_v1.parquet'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
link_dist_vs_lanes[['total_count']].describe()

,total_count
count,5.273930e+05
mean,7.201027e+03
std,2.447247e+04
min,1.000000e+00
25%,4.010000e+02
50%,2.066000e+03
75%,6.302000e+03
max,2.106563e+06


In [ ]:
out = link_dist_vs_lanes[link_dist_vs_lanes['total_count'] > 1000]

In [ ]:
out.shape

(331251, 44)

In [ ]:
out2 = out.sample(n=50000)[link_dist_vs_lanes['lanes_int'] <= 7]

/tmp/ipython-input-128326313.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  out2 = out.sample(n=50000)[link_dist_vs_lanes['lanes_int'] <= 7]


In [ ]:
out2 = out[link_dist_vs_lanes['lanes_int'] <= 7]

/tmp/ipython-input-788360983.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  out2 = out[link_dist_vs_lanes['lanes_int'] <= 7]


In [ ]:
out2.shape

(329787, 44)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------------------------------------------------
# 0. Load data (uncomment if needed)
# -------------------------------------------------------------------
# link_dist_vs_lanes = pd.read_parquet("/content/la_link_dist_vs_lanes_estimation_dataset_v1.parquet")

# -------------------------------------------------------------------
# 1. Filter to out and out2
# -------------------------------------------------------------------
# Keep only links with enough GPS points
out = link_dist_vs_lanes[link_dist_vs_lanes["total_count"] > 1000].copy()

# Keep only reasonable lane counts (≤ 7 lanes)
out2 = out[out["lanes_int"] <= 7].copy()

print("out2 shape:", out2.shape)

# -------------------------------------------------------------------
# 2. Features & target
# -------------------------------------------------------------------
ratio_cols = [str(i) for i in range(40)]   # "0"..."39"
feature_cols = ratio_cols + ["oneway"]

X = out2[feature_cols].copy()
X["oneway"] = X["oneway"].astype(int)      # True/False -> 1/0

y = out2["lanes_int"].astype(int)

# Safety: drop rows with NaNs / inf in X or missing y
mask = np.isfinite(X.values).all(axis=1) & y.notna()
X = X[mask]
y = y[mask]

print("X shape after cleaning:", X.shape)
print("y shape after cleaning:", y.shape)

# -------------------------------------------------------------------
# 3. Train / test split
# -------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape[0], "Test size:", X_test.shape[0])

# -------------------------------------------------------------------
# 4. Random Forest model
# -------------------------------------------------------------------
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced_subsample"
)

rf.fit(X_train, y_train)

# -------------------------------------------------------------------
# 5. Evaluation
# -------------------------------------------------------------------
y_pred = rf.predict(X_test)

print("\n=== Classification Report (Random Forest) ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix (Random Forest) ===")
print(confusion_matrix(y_test, y_pred))

# -------------------------------------------------------------------
# 6. Build results DataFrame (keep original indices!)
# -------------------------------------------------------------------
rf_results = pd.DataFrame(
    {
        "true_lanes": y_test,
        "pred_lanes": y_pred
    },
    index=y_test.index  # ensure index matches out2 rows
)

rf_results["error"] = rf_results["pred_lanes"] - rf_results["true_lanes"]
rf_results["abs_error"] = rf_results["error"].abs()

print("\n=== Sample Predictions (true vs predicted) ===")
display(rf_results.head(10))

# -------------------------------------------------------------------
# 7. Error summaries
# -------------------------------------------------------------------
print("\n=== Error Summary (Absolute Error Counts) ===")
error_counts = rf_results["abs_error"].value_counts().sort_index()
print(error_counts)

print("\n=== Error Summary (Absolute Error Percentages) ===")
error_percentages = (
    rf_results["abs_error"].value_counts(normalize=True).sort_index() * 100
)
print(error_percentages)

# -------------------------------------------------------------------
# 8. Join back with out2 to get FULL RECORDS
# -------------------------------------------------------------------
rf_full = out2.join(rf_results, how="inner")

print("\nrf_full shape (rows with predictions):", rf_full.shape)
print("rf_full columns:", rf_full.columns.tolist())

# Buckets with full records
correct_full      = rf_full[rf_full["abs_error"] == 0]
off_by_1_full     = rf_full[rf_full["abs_error"] == 1]
off_by_2_full     = rf_full[rf_full["abs_error"] == 2]
off_by_3plus_full = rf_full[rf_full["abs_error"] >= 3]

print("\n=== Breakdown by Error Bucket (row counts, full records) ===")
print("Correct predictions (0 lanes off):", correct_full.shape[0])
print("Off by 1 lane:", off_by_1_full.shape[0])
print("Off by 2 lanes:", off_by_2_full.shape[0])
print("Off by 3+ lanes:", off_by_3plus_full.shape[0])

# -------------------------------------------------------------------
# 9. Export CSVs for each bucket (full records)
# -------------------------------------------------------------------
correct_full.to_csv("/content/rf_correct_lanes_full.csv", index=False)
off_by_1_full.to_csv("/content/rf_off_by_1_lane_full.csv", index=False)
off_by_2_full.to_csv("/content/rf_off_by_2_lanes_full.csv", index=False)
off_by_3plus_full.to_csv("/content/rf_off_by_3plus_lanes_full.csv", index=False)

print("\nCSV files saved to /content/:")
print(" - rf_correct_lanes_full.csv")
print(" - rf_off_by_1_lane_full.csv")
print(" - rf_off_by_2_lanes_full.csv")
print(" - rf_off_by_3plus_lanes_full.csv")


out2 shape: (329787, 44)
X shape after cleaning: (329787, 41)
y shape after cleaning: (329787,)
Train size: 263829 Test size: 65958

=== Classification Report (Random Forest) ===
              precision    recall  f1-score   support

           1       0.84      0.72      0.78      3203
           2       0.89      0.97      0.93     24857
           3       0.86      0.73      0.79      8565
           4       0.91      0.92      0.92     14428
           5       0.94      0.91      0.92      9794
           6       0.95      0.88      0.91      3158
           7       0.98      0.89      0.93      1953

    accuracy                           0.90     65958
   macro avg       0.91      0.86      0.88     65958
weighted avg       0.90      0.90      0.90     65958


=== Confusion Matrix (Random Forest) ===
[[ 2322   881     0     0     0     0     0]
 [  424 24164   230    39     0     0     0]
 [   10  1944  6216   363    32     0     0]
 [    0   189   685 13339   192    23     0]
 [

,true_lanes,pred_lanes,error,abs_error
519162,4,4,0,0
75759,5,5,0,0
503283,2,2,0,0
126587,2,2,0,0
245691,2,2,0,0
307207,2,2,0,0
16719,3,3,0,0
272942,2,2,0,0
41486,5,5,0,0
174222,3,2,-1,1



=== Error Summary (Absolute Error Counts) ===
abs_error
0    59438
1     5858
2      655
3        7
Name: count, dtype: int64

=== Error Summary (Absolute Error Percentages) ===
abs_error
0    90.114922
1     8.881409
2     0.993056
3     0.010613
Name: proportion, dtype: float64

rf_full shape (rows with predictions): (65958, 48)
rf_full columns: ['our_id_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 'total_count', 'lanes_int', 'oneway', 'true_lanes', 'pred_lanes', 'error', 'abs_error']

=== Breakdown by Error Bucket (row counts, full records) ===
Correct predictions (0 lanes off): 59438
Off by 1 lane: 5858
Off by 2 lanes: 655
Off by 3+ lanes: 7

CSV files saved to /content/:
 - rf_correct_lanes_full.csv
 - rf_off_by_1_lane_full.csv
 - rf_off_by_2_lanes_full.csv
 - rf_off_by_3plus_lanes_full.csv


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# --- assumes X, y, X_train, X_test, y_train, y_test already exist from RF section ---

# 1) Compute class weights on y_train (to handle imbalance)
classes = np.sort(y_train.unique())
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_map = {c: w for c, w in zip(classes, weights)}
sw_train = y_train.map(class_weight_map).astype(float).values

# 2) Wrap into DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, weight=sw_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# 3) XGBoost params for multiclass lane prediction
params = {
    "objective": "multi:softprob",
    "num_class": int(y.nunique()),          # number of distinct lane counts
    "eval_metric": ["mlogloss", "merror"],
    "tree_method": "hist",                  # or "gpu_hist" if GPU available
    "eta": 0.07,
    "max_depth": 8,
    "min_child_weight": 50,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda": 1.0,
    "seed": 42
}

# 4) Training with early stopping
evals = [(dtrain, "train"), (dtest, "eval")]
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=50
)

# 5) Predict
y_pred_xgb = np.argmax(bst.predict(dtest), axis=1)

print("\n=== XGBoost Classification Report ===")
print(classification_report(y_test, y_pred_xgb, digits=3))

print("\n=== XGBoost Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_xgb))

print("\nBest iteration:", bst.best_iteration)

# 6) Optional: same style error analysis as RF
xgb_results = pd.DataFrame({
    "true_lanes": y_test.values,
    "pred_lanes": y_pred_xgb
})
xgb_results["error"] = xgb_results["pred_lanes"] - xgb_results["true_lanes"]
xgb_results["abs_error"] = xgb_results["error"].abs()

print("\nXGBoost – Error Counts:")
print(xgb_results["abs_error"].value_counts().sort_index())

print("\nXGBoost – Error Percentages:")
print(xgb_results["abs_error"].value_counts(normalize=True).sort_index() * 100)

print("\nSample XGBoost predictions (true vs predicted):")
display(xgb_results.head(10))


[0]	train-mlogloss:1.89150	train-merror:0.18233	eval-mlogloss:1.91520	eval-merror:0.22854
[50]	train-mlogloss:0.43102	train-merror:0.14523	eval-mlogloss:0.55804	eval-merror:0.19035
[100]	train-mlogloss:0.34336	train-merror:0.13272	eval-mlogloss:0.46964	eval-merror:0.18303
[150]	train-mlogloss:0.31609	train-merror:0.12297	eval-mlogloss:0.44773	eval-merror:0.17851
[200]	train-mlogloss:0.29728	train-merror:0.11381	eval-mlogloss:0.43566	eval-merror:0.17446
[250]	train-mlogloss:0.28246	train-merror:0.10646	eval-mlogloss:0.42681	eval-merror:0.17153
[300]	train-mlogloss:0.26970	train-merror:0.09982	eval-mlogloss:0.41957	eval-merror:0.16890
[350]	train-mlogloss:0.25931	train-merror:0.09470	eval-mlogloss:0.41349	eval-merror:0.16670
[400]	train-mlogloss:0.24947	train-merror:0.08986	eval-mlogloss:0.40788	eval-merror:0.16423
[450]	train-mlogloss:0.24100	train-merror:0.08569	eval-mlogloss:0.40287	eval-merror:0.16178
[500]	train-mlogloss:0.23278	train-merror:0.08152	eval-mlogloss:0.39821	eval-merror

,true_lanes,pred_lanes,error,abs_error
0,4,4,0,0
1,5,5,0,0
2,2,2,0,0
3,2,1,-1,1
4,2,2,0,0
5,2,2,0,0
6,3,3,0,0
7,2,2,0,0
8,5,5,0,0
9,3,2,-1,1


In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture


In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------------------------------------------------
# 10. Helper: build (distances, counts) per link from histogram
# -------------------------------------------------------------------
ratio_cols = [str(i) for i in range(40)]

def make_hist_for_link(row):
    """
    For one row of out2:
    - distances: bin indices 0..39 (as floats)
    - counts:    ratio * total_count
    """
    ratios = row[ratio_cols].values.astype(float)
    total = float(row["total_count"]) if "total_count" in row else 1.0

    counts = ratios * total

    # keep only non-zero bins
    mask = counts > 0
    distances = np.arange(len(ratio_cols))[mask].astype(float)
    counts = counts[mask].astype(float)

    return distances, counts

# -------------------------------------------------------------------
# 11. Fast GMM lane counter (unsupervised baseline)
# -------------------------------------------------------------------
def gmm_lane_count(distances, counts=None, max_components=5, max_rep_per_bin=20):
    """
    Fast GMM lane counter.

    distances : 1D np.array of bin positions (0..39)
    counts    : sample weights (float). We approximate weights by
                limiting repetitions per bin to max_rep_per_bin.

    Uses the *shape* of the histogram, not absolute counts.
    """
    if counts is None:
        X = distances.reshape(-1, 1)
    else:
        counts = np.asarray(counts, dtype=float)

        if counts.sum() == 0:
            X = distances.reshape(-1, 1)
        else:
            # normalize to [0, 1]
            weights = counts / counts.max()

            # map to 1..max_rep_per_bin integer repetitions
            counts_int = np.maximum(
                1,
                np.round(weights * max_rep_per_bin).astype(int)
            )

            # bounded-size sample set
            X = np.repeat(distances, counts_int).reshape(-1, 1)

    best_k = 1
    best_score = np.inf

    for K in range(1, max_components + 1):
        gm = GaussianMixture(
            n_components=K,
            covariance_type="full",
            max_iter=200,
            random_state=42
        )
        gm.fit(X)
        bic = gm.bic(X)

        if bic < best_score:
            best_score = bic
            best_k = K

    return best_k

# -------------------------------------------------------------------
# 12. Apply GMM on the SAME test set as Random Forest
# -------------------------------------------------------------------
gmm_preds = []

for idx in X_test.index:           # X_test, y_test from your RF split
    row = out2.loc[idx]            # out2 is the same filtered dataset

    distances, counts = make_hist_for_link(row)

    if len(distances) < 2:
        k = 1                      # fallback if almost no signal
    else:
        k = gmm_lane_count(distances, counts=counts)

    gmm_preds.append(k)

y_pred_gmm = np.array(gmm_preds)

print("\n=== Classification Report (GMM – lane count from mixture components) ===")
print(classification_report(y_test, y_pred_gmm))

print("\n=== Confusion Matrix (GMM) ===")
print(confusion_matrix(y_test, y_pred_gmm))

# -------------------------------------------------------------------
# 13. Build results DataFrame like rf_results
# -------------------------------------------------------------------
gmm_results = pd.DataFrame(
    {
        "true_lanes": y_test,
        "pred_lanes": y_pred_gmm
    },
    index=y_test.index
)

gmm_results["error"] = gmm_results["pred_lanes"] - gmm_results["true_lanes"]
gmm_results["abs_error"] = gmm_results["error"].abs()

print("\n=== Sample Predictions (GMM: true vs predicted) ===")
display(gmm_results.head(10))

# -------------------------------------------------------------------
# 14. Error summaries (same style as RF)
# -------------------------------------------------------------------
print("\n=== GMM Error Summary (Absolute Error Counts) ===")
gmm_error_counts = gmm_results["abs_error"].value_counts().sort_index()
print(gmm_error_counts)

print("\n=== GMM Error Summary (Absolute Error Percentages) ===")
gmm_error_percentages = (
    gmm_results["abs_error"].value_counts(normalize=True).sort_index() * 100
)
print(gmm_error_percentages)

# -------------------------------------------------------------------
# 15. Join back with out2 to get FULL RECORDS (for mapping / QGIS)
# -------------------------------------------------------------------
gmm_full = out2.join(gmm_results, how="inner")

print("\nGMM_full shape (rows with predictions):", gmm_full.shape)
print("GMM_full columns:", gmm_full.columns.tolist())

gmm_correct_full      = gmm_full[gmm_full["abs_error"] == 0]
gmm_off_by_1_full     = gmm_full[gmm_full["abs_error"] == 1]
gmm_off_by_2_full     = gmm_full[gmm_full["abs_error"] == 2]
gmm_off_by_3plus_full = gmm_full[gmm_full["abs_error"] >= 3]

print("\n=== GMM Breakdown by Error Bucket (row counts, full records) ===")
print("Correct predictions (0 lanes off):", gmm_correct_full.shape[0])
print("Off by 1 lane:", gmm_off_by_1_full.shape[0])
print("Off by 2 lanes:", gmm_off_by_2_full.shape[0])
print("Off by 3+ lanes:", gmm_off_by_3plus_full.shape[0])

# -------------------------------------------------------------------
# 16. Export CSVs for each GMM error bucket (optional)
# -------------------------------------------------------------------
gmm_correct_full.to_csv("/content/gmm_correct_lanes_full.csv", index=False)
gmm_off_by_1_full.to_csv("/content/gmm_off_by_1_lane_full.csv", index=False)
gmm_off_by_2_full.to_csv("/content/gmm_off_by_2_lanes_full.csv", index=False)
gmm_off_by_3plus_full.to_csv("/content/gmm_off_by_3plus_lanes_full.csv", index=False)

print("\nGMM CSV files saved to /content/:")
print(" - gmm_correct_lanes_full.csv")
print(" - gmm_off_by_1_lane_full.csv")
print(" - gmm_off_by_2_lanes_full.csv")
print(" - gmm_off_by_3plus_lanes_full.csv")



=== Classification Report (GMM – lane count from mixture components) ===
              precision    recall  f1-score   support

           1       0.15      0.06      0.09      3203
           2       0.34      0.46      0.39     24857
           3       0.12      0.22      0.16      8565
           4       0.20      0.01      0.02     14428
           5       0.03      0.04      0.03      9794
           6       0.00      0.00      0.00      3158
           7       0.00      0.00      0.00      1953

    accuracy                           0.22     65958
   macro avg       0.12      0.11      0.10     65958
weighted avg       0.20      0.22      0.18     65958


=== Confusion Matrix (GMM) ===
[[  206  1409   229     8  1351     0     0]
 [  462 11544  3373   261  9217     0     0]
 [  176  4758  1887    58  1686     0     0]
 [  352  7705  4296   140  1935     0     0]
 [  111  5418  3702   155   408     0     0]
 [   36  1735  1282    61    44     0     0]
 [   12  1126   769    26  

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,true_lanes,pred_lanes,error,abs_error
519162,4,3,-1,1
75759,5,2,-3,3
503283,2,2,0,0
126587,2,2,0,0
245691,2,5,3,3
307207,2,5,3,3
16719,3,3,0,0
272942,2,5,3,3
41486,5,2,-3,3
174222,3,3,0,0



=== GMM Error Summary (Absolute Error Counts) ===
abs_error
0    14185
1    16490
2    13840
3    16303
4     3966
5     1162
6       12
Name: count, dtype: int64

=== GMM Error Summary (Absolute Error Percentages) ===
abs_error
0    21.506110
1    25.000758
2    20.983050
3    24.717244
4     6.012917
5     1.761727
6     0.018193
Name: proportion, dtype: float64

GMM_full shape (rows with predictions): (65958, 48)
GMM_full columns: ['our_id_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 'total_count', 'lanes_int', 'oneway', 'true_lanes', 'pred_lanes', 'error', 'abs_error']

=== GMM Breakdown by Error Bucket (row counts, full records) ===
Correct predictions (0 lanes off): 14185
Off by 1 lane: 16490
Off by 2 lanes: 13840
Off by 3+ lanes: 21443

GMM CSV files saved to /content/:
 - gmm_correct_lanes_f